In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Desde la ubicacion de los archivos copiamos los codigo de ruta y cargamos las bases de datos
df_2004 = pd.read_stata(r"C:\Users\facun\OneDrive\Documentos\usu_individual_T104.dta")
df_2024 = pd.read_excel(r"C:\Users\facun\OneDrive\Documentos\usu_individual_T124.xlsx.xlsx")

# Estandarizar nombres de columnas a mayúsculas
df_2004.columns = df_2004.columns.str.upper()
df_2024.columns = df_2024.columns.str.upper()

# Ya que la Patagonia es la region elegida buscamos su correcta denominacion y filtramos
df_2004['REGION'] = df_2004['REGION'].astype(str)
df_2024['REGION'] = df_2024['REGION'].astype(str)

df_2004 = df_2004[df_2004['REGION'] == 'Patagónica']
df_2024 = df_2024[df_2024['REGION'] == '43']  

# Agregamos ambas columnas con su correspondiente año
df_2004['ANO4'] = 2004
df_2024['ANO4'] = 2024

# Seleccionar variables comunes elegidas
variables_seleccionadas = [
    'P47T', 'IPCF', 'CH04', 'CH06', 'CH08', 
    'P21', 'CAT_INAC', 'PP04D_COD', 'PP04A', 'TRIMESTRE',
    'REGION', 'AGLOMERADO', 'IDECCFR', 'ANO4', 'ESTADO'
]

# Convertir variables numéricas
numeric_vars = ['CH06', 'IPCF', 'P21', 'P47T']
for var in numeric_vars:
    if var in df_2004.columns:
        df_2004[var] = pd.to_numeric(df_2004[var], errors='coerce')
    if var in df_2024.columns:
        df_2024[var] = pd.to_numeric(df_2024[var], errors='coerce')

# Unimos lo data frames de ambos años trabajados
df_combinado = pd.concat(
    [df_2004[variables_seleccionadas], df_2024[variables_seleccionadas]],
    ignore_index=True
)

# Verificar observaciones por año
print("Conteo de observaciones por año:")
print(df_combinado['ANO4'].value_counts())

# Análisis de valores faltantes
valores_faltantes = df_combinado[variables_seleccionadas].isna().sum()
print("Valores faltantes por variable:")
print(valores_faltantes)

# Agrupar por "ANO4" y calcular los valores faltantes sin incluir la columna de agrupación
faltantes_por_anio = df_combinado.groupby("ANO4", group_keys=False).apply(lambda df: df.isnull().sum())
faltantes_por_anio = faltantes_por_anio.transpose()

# Visualizamos resultados
print("Valores faltantes por año:")
print(faltantes_por_anio)




FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\facun\\OneDrive\\Documentos\\usu_individual_T104.dta'

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Reemplazar los valores de CH04 por etiquetas
df_combinado['CH04'] = df_combinado['CH04'].replace({1: 'Varón', 2: 'Mujer'})

# Agrupar por año y sexo y contar
sexo_agrupado = df_combinado.groupby(['ANO4', 'CH04']).size().unstack()

# Calcular porcentajes por año
sexo_porcentaje = sexo_agrupado.div(sexo_agrupado.sum(axis=1), axis=0) * 100

# Crear gráfico
ax = sexo_porcentaje.plot(kind='bar', figsize=(8, 6), color=['pink', 'blue'], edgecolor='black')

# Estética del gráfico
plt.title('Composición por sexo en la región Patagónica (2004 vs 2024)', fontsize=14)
plt.ylabel('Porcentaje (%)')
plt.xlabel('Año')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.yticks(range(0, 101, 10))  # Porcentajes de 10 en 10
plt.legend(title='Sexo')
plt.tight_layout()

# Agregar etiquetas de porcentaje en cada barra
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='edge', fontsize=10)

# Mostrar gráfico
plt.show()


NameError: name 'df_combinado' is not defined

In [5]:
# Seleccionar variables comunes elegidas (ahora incluyendo CH07 y NIVEL_ED)
variables_seleccionadas = [
    'P47T', 'IPCF', 'CH04', 'CH06', 'CH07', 'CH08',
    'NIVEL_ED', 'P21', 'CAT_INAC', 'PP04D_COD', 'PP04A', 'TRIMESTRE',
    'REGION', 'AGLOMERADO', 'IDECCFR', 'ANO4', 'ESTADO'
]

# Unir los datasets con las nuevas variables seleccionadas
df_combinado = pd.concat(
    [df_2004[variables_seleccionadas], df_2024[variables_seleccionadas]],
    ignore_index=True
)


NameError: name 'df_2004' is not defined

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Diccionario con nombres descriptivos de las variables
nombres_variables = {
    'CH04': 'Sexo',
    'CH06': 'Edad',
    'CH07': 'Estado Civil',
    'CH08': 'Parentesco',
    'NIVEL_ED': 'Nivel Educativo',
    'ESTADO': 'Condición de Actividad',
    'CAT_INAC': 'Categoría de Inactividad',
    'IPCF': 'Ingreso per Cápita Familiar'
}

# Variables a correlacionar
variables_correlacion = list(nombres_variables.keys())

# Asegurarse de que todas las variables sean numéricas
for var in variables_correlacion:
    if var in df_combinado.columns:
        df_combinado[var] = pd.to_numeric(df_combinado[var], errors='coerce')

# Filtrar por año
df_2004_corr = df_combinado[df_combinado['ANO4'] == 2004][variables_correlacion].rename(columns=nombres_variables)
df_2024_corr = df_combinado[df_combinado['ANO4'] == 2024][variables_correlacion].rename(columns=nombres_variables)

# Graficar heatmaps
plt.figure(figsize=(14, 6))

# Año 2004
plt.subplot(1, 2, 1)
sns.heatmap(df_2004_corr.corr(), annot=True, cmap='viridis', fmt=".2f", square=True)
plt.title('Matriz de Correlación - Año 2004')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Año 2024
plt.subplot(1, 2, 2)
sns.heatmap(df_2024_corr.corr(), annot=True, cmap='viridis', fmt=".2f", square=True)
plt.title('Matriz de Correlación - Año 2024')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()


NameError: name 'df_combinado' is not defined

PARTE III


In [17]:
import pandas as pd

# Cargar los datos de los archivos CSV
data_2004 = pd.read_csv('EPH_2004.csv')
data_2024 = pd.read_csv('EPH_2024.csv')

# Ver las primeras filas para asegurarnos de que los datos se importaron correctamente
print(data_2004.head())
print(data_2024.head())


FileNotFoundError: [Errno 2] No such file or directory: 'EPH_2004.csv'

In [ ]:
# Eliminar filas duplicadas
data_2004 = data_2004.drop_duplicates()
data_2024 = data_2024.drop_duplicates()

# Eliminar valores nulos en columnas específicas
data_2004 = data_2004.dropna(subset=['columna_interes'])
data_2024 = data_2024.dropna(subset=['columna_interes'])

# Verificar que no hay más valores nulos
print(data_2004.isnull().sum())
print(data_2024.isnull().sum())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualización de distribución de una columna
sns.histplot(data_2004['columna_interes'], kde=True)
plt.title('Distribución de columna_interes 2004')
plt.show()

# Visualización de una correlación entre dos variables
sns.heatmap(data_2004.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de correlación 2004')
plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Estandarizar las variables numéricas
scaler = StandardScaler()
data_2004[['columna_numérica1', 'columna_numérica2']] = scaler.fit_transform(data_2004[['columna_numérica1', 'columna_numérica2']])

# Codificar variables categóricas
encoder = LabelEncoder()
data_2004['columna_categorica'] = encoder.fit_transform(data_2004['columna_categorica'])


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X = data_2004.drop('columna_objetivo', axis=1)  # Variables independientes
y = data_2004['columna_objetivo']  # Variable dependiente (objetivo)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Selección de modelos
modelo_lr = LinearRegression()
modelo_rf = RandomForestRegressor()

# Ajustar los modelos a los datos de entrenamiento
modelo_lr.fit(X_train, y_train)
modelo_rf.fit(X_train, y_train)


In [ ]:
# Los modelos ya fueron entrenados en el paso anterior
# Aquí podemos predecir utilizando los datos de prueba

predicciones_lr = modelo_lr.predict(X_test)
predicciones_rf = modelo_rf.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluar los modelos
mse_lr = mean_squared_error(y_test, predicciones_lr)
mse_rf = mean_squared_error(y_test, predicciones_rf)

r2_lr = r2_score(y_test, predicciones_lr)
r2_rf = r2_score(y_test, predicciones_rf)

print(f'MSE Regresión Lineal: {mse_lr}')
print(f'MSE Random Forest: {mse_rf}')
print(f'R2 Regresión Lineal: {r2_lr}')
print(f'R2 Random Forest: {r2_rf}')
